<a href="https://www.kaggle.com/code/girishkalyan/bp-prediction?scriptVersionId=222783335" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import warnings

In [3]:
warnings.filterwarnings("ignore")


In [4]:
train = pd.read_csv("/kaggle/input/playground-series-s5e2/train.csv")
train_extra = pd.read_csv("/kaggle/input/playground-series-s5e2/training_extra.csv")
train_data = pd.concat([train, train_extra], axis=0, ignore_index=True)

test_data = pd.read_csv("/kaggle/input/playground-series-s5e2/test.csv")



In [5]:
# Encode categorical features
categorical_features = ['Brand', 'Material', 'Size', 'Style', 'Color']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_encoded = encoder.fit_transform(train_data[categorical_features])
categorical_data = pd.DataFrame(categorical_encoded, index=train_data.index, columns=encoder.get_feature_names_out(categorical_features))


****Feature Extraction****

In [6]:
# Encode categorical features for test data
categorical_encoded_test = encoder.fit_transform(test_data[categorical_features])
categorical_data_test = pd.DataFrame(categorical_encoded_test, columns=encoder.get_feature_names_out(categorical_features))

In [7]:
numerical_features = ['id','Compartments', 'Weight Capacity (kg)']
binary_features = ['Laptop Compartment', 'Waterproof']


In [8]:
test_data[binary_features] = test_data[binary_features].replace({'Yes': 1, 'No': 0})


In [9]:
train_data[binary_features] = train_data[binary_features].replace({'Yes': 1, 'No': 0})
test_data[binary_features] = test_data[binary_features].replace({'Yes': 1, 'No': 0})


In [10]:
X_test = pd.concat([test_data[numerical_features + binary_features], categorical_data_test], axis=1)

In [11]:
X_train = pd.concat([train_data[numerical_features + binary_features], categorical_data], axis=1)
Y_train = train_data[["Price"]]
X_test = pd.concat([test_data[numerical_features + binary_features], categorical_data_test], axis=1)

In [12]:
display(X_train)

,id,Compartments,Weight Capacity (kg),Laptop Compartment,Waterproof,Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,...,Style_Messenger,Style_Tote,Style_nan,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red,Color_nan
0,0,7.0,11.611723,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,10.0,27.078537,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,2.0,16.643760,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,8.0,12.937220,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,1.0,17.749338,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994313,4194313,3.0,28.098120,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3994314,4194314,10.0,17.379531,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3994315,4194315,10.0,17.037708,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3994316,4194316,2.0,28.783339,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [13]:
X_train.drop('id', axis =1)

,Compartments,Weight Capacity (kg),Laptop Compartment,Waterproof,Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Brand_nan,...,Style_Messenger,Style_Tote,Style_nan,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red,Color_nan
0,7.0,11.611723,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.0,27.078537,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2.0,16.643760,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,8.0,12.937220,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,17.749338,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994313,3.0,28.098120,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3994314,10.0,17.379531,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3994315,10.0,17.037708,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3994316,2.0,28.783339,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


****Feature Scaling****

In [14]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Use IterativeImputer (more intelligent but still efficient)
imputer = IterativeImputer(max_iter=10, random_state=42)

X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)

print("Imputation complete using IterativeImputer!")


Imputation complete using IterativeImputer!


In [15]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Use IterativeImputer (more intelligent but still efficient)
imputer = IterativeImputer(max_iter=10, random_state=42)

X_test = pd.DataFrame(imputer.fit_transform(X_test), columns=X_test.columns)

print("Imputation complete using IterativeImputer!")


Imputation complete using IterativeImputer!


In [16]:
X_train.to_csv("/kaggle/working/imputated_train_data.csv", index=False)
X_test.to_csv("/kaggle/working/imputated_test_data.csv", index=False)


In [17]:
import pandas as pd
X_train = pd.read_csv('imputated_train_data.csv')
X_test = pd.read_csv('imputated_test_data.csv')

In [18]:
missing_count_per_column = X_test.isnull().sum()
print(missing_count_per_column)


id                      0
Compartments            0
Weight Capacity (kg)    0
Laptop Compartment      0
Waterproof              0
Brand_Adidas            0
Brand_Jansport          0
Brand_Nike              0
Brand_Puma              0
Brand_Under Armour      0
Brand_nan               0
Material_Canvas         0
Material_Leather        0
Material_Nylon          0
Material_Polyester      0
Material_nan            0
Size_Large              0
Size_Medium             0
Size_Small              0
Size_nan                0
Style_Backpack          0
Style_Messenger         0
Style_Tote              0
Style_nan               0
Color_Black             0
Color_Blue              0
Color_Gray              0
Color_Green             0
Color_Pink              0
Color_Red               0
Color_nan               0
dtype: int64


In [19]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [20]:
# Ensure your data is in NumPy array format
X_train = X_train.to_numpy() if hasattr(X_train, 'to_numpy') else X_train
Y_train = Y_train.to_numpy() if hasattr(Y_train, 'to_numpy') else Y_train
X_test = X_test.to_numpy() if hasattr(X_test, 'to_numpy') else X_test


**Random Forrest Initialization**

In [21]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [22]:
import tensorflow as tf

# Initialize the MirroredStrategy
strategy = tf.distribute.MirroredStrategy()

print(f"Number of devices: {strategy.num_replicas_in_sync}")


Number of devices: 2


In [23]:
with strategy.scope():
    model = RandomForestRegressor(
            n_estimators= 30,            # Number of trees in the forest
            max_depth=None,              # Maximum depth of the tree (None allows full depth)
            min_samples_split=2,         # Minimum samples required to split an internal node
            min_samples_leaf=1,          # Minimum samples required to be a leaf node
            max_features='sqrt',         # Number of features to consider when looking for the best split
            bootstrap=True,              # Whether bootstrap samples are used when building trees
            random_state=42             
        )

In [25]:
pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 60.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 97.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.7/648.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 13.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [27]:
import numpy as np
import pandas as pd
import joblib
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Initialize MLflow
mlflow.set_experiment("RandomForest_Regression")

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Split data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

with mlflow.start_run():
    # Log model parameters
    mlflow.log_param("model_type", "RandomForestRegressor")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)

    # Train the model
    model.fit(X_train, Y_train)

    # Evaluate the model
    Y_pred = model.predict(X_val)
    r2 = r2_score(Y_val, Y_pred)

    print(f"R2 Score: {r2:.4f}")

    # Log performance in MLflow
    mlflow.log_metric("R2_score", r2)

    # Save the trained model
    model_path = "random_forest_model.pkl"
    joblib.dump(model, model_path)
    mlflow.sklearn.log_model(model, "random_forest_model")

    print("Model saved and logged in MLflow.")


R2 Score: -0.0322


OSError: [Errno 28] No space left on device

In [33]:
predictions = model.predict(X_test)
df = pd.DataFrame(predictions, columns=["Predictions"])

# Save DataFrame to an Excel file
file_path = "model_predictions.xlsx"
df.to_excel(file_path, index=False)

print(f"Excel file saved as {file_path}")

OSError: [Errno 28] No space left on device

In [31]:
display(test_data["id"])

0         300000
1         300001
2         300002
3         300003
4         300004
           ...  
199995    499995
199996    499996
199997    499997
199998    499998
199999    499999
Name: id, Length: 200000, dtype: int64